# Cercador model òptim de DNN 

In [1]:
import numpy as np
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot
import plotly.graph_objects as go
import math
import seaborn as sns
from sklearn.metrics import mean_squared_error
from time import time

np.random.seed(2)
tf.random.set_seed(2)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras import backend

C:\Users\dj_kr\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Càrrega de les dades

In [2]:
df = pd.read_csv('data/SentDATA.csv')

df.reset_index(inplace=True)
df['Time'] = pd.to_datetime(df['Time'])
df = df.set_index('Time')

## Anàlisis estadístic

## Transformació de dades

In [3]:
columns = ['PM1','PM25','PM10','PM1ATM','PM25ATM','PM10ATM']

df1 = df.copy();

df1 = df1.rename(columns={"PM 1":"PM1","PM 2.5":"PM25","PM 10":"PM10","PM 1 ATM":"PM1ATM","PM 2.5 ATM":"PM25ATM","PM 10 ATM":"PM10ATM"})

df1['PM1'] = df['PM 1'].astype(np.float32)
df1['PM25'] = df['PM 2.5'].astype(np.float32)
df1['PM10'] = df['PM 10'].astype(np.float32)
df1['PM1ATM'] = df['PM 1 ATM'].astype(np.float32)
df1['PM25ATM'] = df['PM 2.5 ATM'].astype(np.float32)
df1['PM10ATM'] = df['PM 10 ATM'].astype(np.float32)

In [4]:
df2 = df1.copy()

## Crear dades d'entrenament i de test

In [5]:
train_size = int(len(df2) * 0.8)
test_size = len(df2) - train_size
train, test = df2.iloc[0:train_size], df2.iloc[train_size:len(df2)]
train.shape, test.shape

((3991, 7), (998, 7))

## Normalitzar les dades d'entrenament

In [6]:
#Standardize the data
for col in columns:
    scaler = MinMaxScaler()
    train[col] = scaler.fit_transform(train[[col]])


<ipython-input-6-c8a1383fd1da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[col] = scaler.fit_transform(train[[col]])
<ipython-input-6-c8a1383fd1da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[col] = scaler.fit_transform(train[[col]])
<ipython-input-6-c8a1383fd1da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

## Crear finestra de temps PM 2.5

In [7]:
TIME_STEPS=144 #6 registres hora x 24h x 3 --> equival a una finestra d'un dia

def create_sequences(X, y, time_steps=TIME_STEPS):
    Xs, ys = [], []
    for i in range(len(X)-time_steps):
        Xs.append(X.iloc[i:(i+time_steps)].values)
        ys.append(y.iloc[i+time_steps])
    
    return np.array(Xs), np.array(ys)

X_train1h, y_train1h = create_sequences(train[[columns[1]]], train[columns[1]], 6) #1 hour

X_train3h, y_train3h = create_sequences(train[[columns[1]]], train[columns[1]], 18) #3 hours

X_train6h, y_train6h = create_sequences(train[[columns[1]]], train[columns[1]], 36) #6 hours

X_train12h, y_train12h = create_sequences(train[[columns[1]]], train[columns[1]], 72) #12 hours

X_train1d, y_train1d = create_sequences(train[[columns[1]]], train[columns[1]], 144) #1 day

X_train3d, y_train3d = create_sequences(train[[columns[1]]], train[columns[1]], 432) #3 days

X_train7d, y_train7d = create_sequences(train[[columns[1]]], train[columns[1]], 1008) #7 days
#X_test, y_test = create_sequences(test[[columns[1]]], test[columns[1]])

print(f'X_train1h shape: {X_train1d.shape}')
print(f'X_train3d shape: {X_train3h.shape}')
print(f'X_train6h shape: {X_train6h.shape}')
print(f'X_train12h shape: {X_train12h.shape}')
print(f'X_train1d shape: {X_train1d.shape}')
print(f'X_train3d shape: {X_train3d.shape}')
print(f'X_train7d shape: {X_train7d.shape}')

X_train1h shape: (3847, 144, 1)
X_train3d shape: (3973, 18, 1)
X_train6h shape: (3955, 36, 1)
X_train12h shape: (3919, 72, 1)
X_train1d shape: (3847, 144, 1)
X_train3d shape: (3559, 432, 1)
X_train7d shape: (2983, 1008, 1)


In [8]:
def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()

    print(model_name + ':')
    print('Mean Absolute Error: {:.4f}'.format(mae))
    print('Root Mean Square Error: {:.4f}'.format(rmse))
    print('Mean Square Error: {:.4f}'.format(mse))
    print('')
    return mae,rmse,mse

## Cerca dels models òptims

In [9]:
def _model(units,activationDense,dropout1,optimizer):
    model = Sequential()
    model.add(Dense(units=units, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(rate=dropout1))
    model.add(Dense(X_train.shape[2],activation=activationDense))
    model.compile(optimizer=optimizer, loss='mae')
    model.summary()
    
    return model

sequences = ["1h","3h","6h","12h","1d", "3d", "7d"]
X_trains = [X_train1h,X_train3h, X_train6h, X_train12h,X_train1d,X_train3d, X_train7d]
y_trains= [y_train1h,y_train3h, y_train6h,y_train12h,y_train1d,y_train3d, y_train7d]
activationsDense = ['tanh','sigmoid']
optimizers = ['adam','adadelta','adamax']
list_validationSplit = [0.1,0.2]
list_dropout1 =  np.random.uniform(0.1,0.8,5)
list_units = np.random.randint(5,high=100, size=5)
list_epochs = np.random.randint(5,high=100, size=5) 
list_batchsize = np.random.randint(5,high=64, size=5)   

list_results = pd.DataFrame()

for X_train, y_train,sequence in zip(X_trains,y_trains,sequences):
    for optimizer in optimizers:
        #for activation in activations:
            for activationDense in activationsDense:
                for validationsplit in list_validationSplit: 
                    for units,epochs,batchsize,dropout1 in zip(list_units,list_epochs,list_batchsize,list_dropout1): 
                        start = time()
                        print("###########################\n")
                        print("MODEL: ", "DNN")
                        print('sequence: ',sequence)
                        print('units: ',units)
                        print('dropout1: ',dropout1)
                        print('optimizer:',optimizer)
                        print('activationDense:',activationDense)
                        print('epochs:',epochs)
                        print('batchsize:',batchsize)
                        print('validation_split:',validationsplit)

                        model = _model(units,activationDense,dropout1,optimizer)
                        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batchsize, validation_split=validationsplit, shuffle=False)
                        end = time()
                        totalTime = end-start
                        print ('Execution time: ',totalTime)

                        X_train_pred = model.predict(X_train, verbose=0)

                        mae,rmse,mse = evaluate_prediction(X_train_pred, X_train,"DNN")

                        print('Train RMSE: %.3f' % rmse);
                        print('Train MSE: %.3f' % mse);
                        print('Train MAE: %.3f' % mae);

                        result = pd.DataFrame({
                                               #'activation':[activation],
                                               'model':["DNN"],
                                               'sequence':[sequence],
                                               'activationDense':[activationDense],
                                               'optimizer':[optimizer],
                                               'dropout1':[dropout1],
                                               'units':[units],
                                               'epochs':[epochs],
                                               'batchsize':[batchsize],
                                               'validation_split':[validationsplit],

                                               'RMSE':[rmse],
                                               'MSE':[mse],
                                               'MAE':[mae],                            
                                               'Time':[totalTime]})
                        list_results = list_results.append(result) 

###########################

MODEL:  DNN
sequence:  1h
units:  87
dropout1:  0.40519643149940265
optimizer: adam
activationDense: tanh
epochs: 80
batchsize: 31
validation_split: 0.1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6, 87)             174       
_________________________________________________________________
dense_1 (Dense)              (None, 6, 16)             1408      
_________________________________________________________________
dropout (Dropout)            (None, 6, 16)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 6, 1)              17        
Total params: 1,599
Trainable params: 1,599
Non-trainable params: 0
_________________________________________________________________
Epoch 1/80
116/116 [==============================] - 0s 4ms/step - lo

116/116 [==============================] - 0s 2ms/step - loss: 0.0149 - val_loss: 0.0134
Epoch 72/80
116/116 [==============================] - 0s 2ms/step - loss: 0.0161 - val_loss: 0.0158
Epoch 73/80
116/116 [==============================] - 0s 2ms/step - loss: 0.0167 - val_loss: 0.0130
Epoch 74/80
116/116 [==============================] - 0s 2ms/step - loss: 0.0169 - val_loss: 0.0136
Epoch 75/80
116/116 [==============================] - 0s 2ms/step - loss: 0.0164 - val_loss: 0.0154
Epoch 76/80
116/116 [==============================] - 0s 2ms/step - loss: 0.0161 - val_loss: 0.0149
Epoch 77/80
116/116 [==============================] - 0s 2ms/step - loss: 0.0154 - val_loss: 0.0172
Epoch 78/80
116/116 [==============================] - 0s 2ms/step - loss: 0.0153 - val_loss: 0.0156
Epoch 79/80
116/116 [==============================] - 0s 2ms/step - loss: 0.0154 - val_loss: 0.0173
Epoch 80/80
116/116 [==============================] - 0s 2ms/step - loss: 0.0159 - val_loss: 0.0149
Ex

144/144 [==============================] - 0s 2ms/step - loss: 0.0109 - val_loss: 0.0149
Epoch 61/90
144/144 [==============================] - 0s 2ms/step - loss: 0.0107 - val_loss: 0.0124
Epoch 62/90
144/144 [==============================] - 0s 2ms/step - loss: 0.0111 - val_loss: 0.0147
Epoch 63/90
144/144 [==============================] - 0s 2ms/step - loss: 0.0110 - val_loss: 0.0141
Epoch 64/90
144/144 [==============================] - 0s 2ms/step - loss: 0.0105 - val_loss: 0.0122
Epoch 65/90
144/144 [==============================] - 0s 1ms/step - loss: 0.0109 - val_loss: 0.0104
Epoch 66/90
144/144 [==============================] - 0s 2ms/step - loss: 0.0112 - val_loss: 0.0104
Epoch 67/90
144/144 [==============================] - 0s 2ms/step - loss: 0.0110 - val_loss: 0.0120
Epoch 68/90
144/144 [==============================] - 0s 2ms/step - loss: 0.0109 - val_loss: 0.0146
Epoch 69/90
144/144 [==============================] - 0s 2ms/step - loss: 0.0108 - val_loss: 0.0126
Ep

Epoch 40/52
63/63 [==============================] - 0s 2ms/step - loss: 0.0163 - val_loss: 0.0115
Epoch 41/52
63/63 [==============================] - 0s 2ms/step - loss: 0.0168 - val_loss: 0.0098
Epoch 42/52
63/63 [==============================] - 0s 2ms/step - loss: 0.0155 - val_loss: 0.0114
Epoch 43/52
63/63 [==============================] - 0s 2ms/step - loss: 0.0160 - val_loss: 0.0096
Epoch 44/52
63/63 [==============================] - 0s 2ms/step - loss: 0.0154 - val_loss: 0.0125
Epoch 45/52
63/63 [==============================] - 0s 2ms/step - loss: 0.0155 - val_loss: 0.0098
Epoch 46/52
63/63 [==============================] - 0s 2ms/step - loss: 0.0150 - val_loss: 0.0123
Epoch 47/52
63/63 [==============================] - 0s 2ms/step - loss: 0.0152 - val_loss: 0.0100
Epoch 48/52
63/63 [==============================] - 0s 2ms/step - loss: 0.0152 - val_loss: 0.0128
Epoch 49/52
63/63 [==============================] - 0s 2ms/step - loss: 0.0158 - val_loss: 0.0096
Epoch 50/5

103/103 [==============================] - 0s 2ms/step - loss: 0.0146 - val_loss: 0.0114
Epoch 58/80
103/103 [==============================] - 0s 2ms/step - loss: 0.0145 - val_loss: 0.0118
Epoch 59/80
103/103 [==============================] - 0s 2ms/step - loss: 0.0147 - val_loss: 0.0118
Epoch 60/80
103/103 [==============================] - 0s 2ms/step - loss: 0.0146 - val_loss: 0.0112
Epoch 61/80
103/103 [==============================] - 0s 2ms/step - loss: 0.0143 - val_loss: 0.0108
Epoch 62/80
103/103 [==============================] - 0s 2ms/step - loss: 0.0139 - val_loss: 0.0110
Epoch 63/80
103/103 [==============================] - 0s 2ms/step - loss: 0.0144 - val_loss: 0.0117
Epoch 64/80
103/103 [==============================] - 0s 2ms/step - loss: 0.0146 - val_loss: 0.0108
Epoch 65/80
103/103 [==============================] - 0s 2ms/step - loss: 0.0143 - val_loss: 0.0120
Epoch 66/80
103/103 [==============================] - 0s 2ms/step - loss: 0.0147 - val_loss: 0.0120
Ep

...

108/108 [==============================] - 4s 34ms/step - loss: 0.0419 - val_loss: 0.0233
Epoch 2/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0287 - val_loss: 0.0219
Epoch 3/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0286 - val_loss: 0.0214
Epoch 4/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0284 - val_loss: 0.0212
Epoch 5/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0282 - val_loss: 0.0207
Epoch 6/90
108/108 [==============================] - 3s 29ms/step - loss: 0.0280 - val_loss: 0.0205
Epoch 7/90
108/108 [==============================] - 3s 28ms/step - loss: 0.0279 - val_loss: 0.0203
Epoch 8/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0277 - val_loss: 0.0201
Epoch 9/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0276 - val_loss: 0.0199
Epoch 10/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0275 - val_loss: 0.0203


108/108 [==============================] - 3s 28ms/step - loss: 0.0233 - val_loss: 0.0146
Epoch 82/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0233 - val_loss: 0.0147
Epoch 83/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0233 - val_loss: 0.0147
Epoch 84/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0233 - val_loss: 0.0147
Epoch 85/90
108/108 [==============================] - 3s 28ms/step - loss: 0.0233 - val_loss: 0.0148
Epoch 86/90
108/108 [==============================] - 3s 29ms/step - loss: 0.0233 - val_loss: 0.0148
Epoch 87/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0232 - val_loss: 0.0148
Epoch 88/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0232 - val_loss: 0.0139
Epoch 89/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0233 - val_loss: 0.0149
Epoch 90/90
108/108 [==============================] - 3s 27ms/step - loss: 0.0232 - val_loss:

Epoch 1/80
77/77 [==============================] - 3s 39ms/step - loss: 0.0544 - val_loss: 0.0251
Epoch 2/80
77/77 [==============================] - 3s 36ms/step - loss: 0.0404 - val_loss: 0.0224
Epoch 3/80
77/77 [==============================] - 3s 37ms/step - loss: 0.0385 - val_loss: 0.0198
Epoch 4/80
77/77 [==============================] - 3s 36ms/step - loss: 0.0368 - val_loss: 0.0160
Epoch 5/80
77/77 [==============================] - 3s 37ms/step - loss: 0.0353 - val_loss: 0.0154
Epoch 6/80
77/77 [==============================] - 3s 39ms/step - loss: 0.0346 - val_loss: 0.0154
Epoch 7/80
77/77 [==============================] - 3s 37ms/step - loss: 0.0340 - val_loss: 0.0151
Epoch 8/80
77/77 [==============================] - 3s 37ms/step - loss: 0.0336 - val_loss: 0.0148
Epoch 9/80
77/77 [==============================] - 3s 37ms/step - loss: 0.0332 - val_loss: 0.0146
Epoch 10/80
77/77 [==============================] - 3s 37ms/step - loss: 0.0328 - val_loss: 0.0145
Epoch 11/

96/96 [==============================] - 3s 30ms/step - loss: 0.0404 - val_loss: 0.0147
Epoch 2/90
96/96 [==============================] - 3s 30ms/step - loss: 0.0301 - val_loss: 0.0135
Epoch 3/90
96/96 [==============================] - 3s 29ms/step - loss: 0.0297 - val_loss: 0.0130
Epoch 4/90
96/96 [==============================] - 3s 28ms/step - loss: 0.0295 - val_loss: 0.0129
Epoch 5/90
96/96 [==============================] - 3s 28ms/step - loss: 0.0293 - val_loss: 0.0126
Epoch 6/90
96/96 [==============================] - 3s 28ms/step - loss: 0.0292 - val_loss: 0.0126
Epoch 7/90
96/96 [==============================] - 3s 28ms/step - loss: 0.0288 - val_loss: 0.0124
Epoch 8/90
96/96 [==============================] - 3s 30ms/step - loss: 0.0288 - val_loss: 0.0121
Epoch 9/90
96/96 [==============================] - 3s 29ms/step - loss: 0.0287 - val_loss: 0.0120
Epoch 10/90
96/96 [==============================] - 3s 28ms/step - loss: 0.0285 - val_loss: 0.0119
Epoch 11/90
96/96 [=

96/96 [==============================] - 3s 29ms/step - loss: 0.0252 - val_loss: 0.0126
Epoch 84/90
96/96 [==============================] - 3s 28ms/step - loss: 0.0252 - val_loss: 0.0127
Epoch 85/90
96/96 [==============================] - 3s 29ms/step - loss: 0.0252 - val_loss: 0.0127
Epoch 86/90
96/96 [==============================] - 3s 28ms/step - loss: 0.0252 - val_loss: 0.0126
Epoch 87/90
96/96 [==============================] - 3s 28ms/step - loss: 0.0251 - val_loss: 0.0120
Epoch 88/90
96/96 [==============================] - 3s 30ms/step - loss: 0.0252 - val_loss: 0.0129
Epoch 89/90
96/96 [==============================] - 3s 29ms/step - loss: 0.0251 - val_loss: 0.0126
Epoch 90/90
96/96 [==============================] - 3s 28ms/step - loss: 0.0251 - val_loss: 0.0126
Execution time:  252.21988654136658
DNN:
Mean Absolute Error: 0.0261
Root Mean Square Error: 0.0469
Mean Square Error: 0.0022

Train RMSE: 0.047
Train MSE: 0.002
Train MAE: 0.026
###########################

MODE

In [10]:
list_results.to_csv("resultats-cerca-optim-dnn-v1.csv")

In [11]:
list_results.head(20)

,model,sequence,activationDense,optimizer,dropout1,units,epochs,batchsize,validation_split,RMSE,MSE,MAE,Time
0,DNN,1h,tanh,adam,0.405196,87,80,31,0.1,0.019200,0.000369,0.009905,17.237772
0,DNN,1h,tanh,adam,0.118148,80,90,25,0.1,0.018856,0.000356,0.005032,22.277235
0,DNN,1h,tanh,adam,0.484764,12,52,57,0.1,0.017678,0.000313,0.011355,6.903105
0,DNN,1h,tanh,adam,0.405196,87,80,31,0.2,0.021332,0.000455,0.013577,16.152798
0,DNN,1h,tanh,adam,0.118148,80,90,25,0.2,0.021047,0.000443,0.008638,20.035282
0,DNN,1h,tanh,adam,0.484764,12,52,57,0.2,0.024989,0.000624,0.016478,6.411896
0,DNN,1h,sigmoid,adam,0.405196,87,80,31,0.1,0.018717,0.000350,0.010021,16.140609
0,DNN,1h,sigmoid,adam,0.118148,80,90,25,0.1,0.020069,0.000403,0.005485,20.425318
0,DNN,1h,sigmoid,adam,0.484764,12,52,57,0.1,0.019351,0.000374,0.010890,6.711839
0,DNN,1h,sigmoid,adam,0.405196,87,80,31,0.2,0.028214,0.000796,0.014225,15.500949


In [12]:
list_results.sort_values(by=['RMSE', 'sequence'])

,model,sequence,activationDense,optimizer,dropout1,units,epochs,batchsize,validation_split,RMSE,MSE,MAE,Time
0,DNN,1h,sigmoid,adamax,0.118148,80,90,25,0.2,0.013449,0.000181,0.005524,19.100332
0,DNN,1h,sigmoid,adamax,0.405196,87,80,31,0.2,0.013451,0.000181,0.007040,15.485010
0,DNN,1h,tanh,adamax,0.405196,87,80,31,0.1,0.013571,0.000184,0.007138,15.828002
0,DNN,1h,tanh,adamax,0.484764,12,52,57,0.1,0.013672,0.000187,0.007954,6.807474
0,DNN,1h,tanh,adamax,0.118148,80,90,25,0.2,0.014728,0.000217,0.009229,19.297134
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,DNN,7d,tanh,adadelta,0.484764,12,52,57,0.2,0.371034,0.137667,0.368752,74.603660
0,DNN,12h,tanh,adadelta,0.484764,12,52,57,0.2,0.380825,0.145028,0.378488,10.224256
0,DNN,1d,tanh,adadelta,0.484764,12,52,57,0.1,0.443665,0.196839,0.440807,17.952344
0,DNN,6h,tanh,adadelta,0.484764,12,52,57,0.2,0.452307,0.204582,0.449499,9.281137
